<a href="https://colab.research.google.com/github/divyasrimandadi/ExcelR-Training/blob/main/cosine_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nltk
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
import warnings
warnings.filterwarnings('ignore')

In [5]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
#movie reviews
review1="The movie was amazing.The acting of hero was Fabulous"
review2="The movie was boring.I hate the acting of villain."

In [7]:
def preprocess_text(text):
  tokens=word_tokenize(text)
  tokens=[word.lower() for word in tokens]
  stop_words=set(stopwords.words('english'))
  tokens=[word for word in tokens if word not in stop_words]
  lemmatizer=WordNetLemmatizer()
  tokens=[lemmatizer.lemmatize(word) for word in tokens]
  return ' '.join(tokens)

In [8]:
a=preprocess_text(review1)
print(a)
b=preprocess_text(review2)
print(b)

movie amazing.the acting hero fabulous
movie boring.i hate acting villain .


In [9]:
vec=TfidfVectorizer()
X=vec.fit_transform([a,b])
X.toarray()

array([[0.31779954, 0.44665616, 0.        , 0.44665616, 0.        ,
        0.44665616, 0.31779954, 0.44665616, 0.        ],
       [0.35520009, 0.        , 0.49922133, 0.        , 0.49922133,
        0.        , 0.35520009, 0.        , 0.49922133]])

In [10]:
X[0].toarray()

array([[0.31779954, 0.44665616, 0.        , 0.44665616, 0.        ,
        0.44665616, 0.31779954, 0.44665616, 0.        ]])

In [11]:
cosine_sim=cosine_similarity(X[0],X[1])[0][0]
print(round(cosine_sim,3))

0.226


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/train.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [14]:
df.shape

(27481, 4)

In [15]:
df["text1_clean"]=df["text"].astype(str).apply(preprocess_text)
df["text2_clean"]=df["selected_text"].astype(str).apply(preprocess_text)

In [16]:
df.head()

,textID,text,selected_text,sentiment,text1_clean,text2_clean
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"` responded , going","` responded , going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego ! ! !,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,bos bullying ...,bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview ! leave alone,leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"son * * * * , ` put release already bought","son * * * * ,"


In [17]:
vect=TfidfVectorizer()
m1=vect.fit_transform(df["text1_clean"])
m2=vect.transform(df["text2_clean"])

In [18]:
cosine_=cosine_similarity(m1,m2)
df['cosine_similarities']=cosine_.diagonal()
df.head()

,textID,text,selected_text,sentiment,text1_clean,text2_clean,cosine_similarities
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"` responded , going","` responded , going",1.000000
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego ! ! !,sooo sad,0.535921
2,088c60f138,my boss is bullying me...,bullying me,negative,bos bullying ...,bullying,0.791472
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview ! leave alone,leave alone,0.763914
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"son * * * * , ` put release already bought","son * * * * ,",0.473126


In [19]:
def jaccard_similarity(text1, text2):
  t1=set(text1.split())
  t2=set(text2.split())
  intersection=len(t1.intersection(t2))
  union=t1.union(t2)
  if len(union)==0:
    return 0
  else:
    return intersection/len(union)

In [20]:
df['jaccard_similarity']=df.apply(lambda x:jaccard_similarity(x['text1_clean'],x['text2_clean']),axis=1)
df.head()

,textID,text,selected_text,sentiment,text1_clean,text2_clean,cosine_similarities,jaccard_similarity
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"` responded , going","` responded , going",1.000000,1.000000
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego ! ! !,sooo sad,0.535921,0.333333
2,088c60f138,my boss is bullying me...,bullying me,negative,bos bullying ...,bullying,0.791472,0.333333
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview ! leave alone,leave alone,0.763914,0.500000
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"son * * * * , ` put release already bought","son * * * * ,",0.473126,0.375000
